<a href="https://colab.research.google.com/github/HassanAlam55/ARGOG_COLAB/blob/main/VectorDB_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating Vector DB for ARGOG Paper

In [1]:
# install packages if they dont exist:
import subprocess
import sys

# Function to install a package if not already installed
def install_package(package_name, import_name=None):
    try:
        if import_name is None:
            import_name = package_name
        __import__(import_name)
    except ImportError:
        print(f"{package_name} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
    else:
        print(f"{package_name} is already installed.")

# List of packages to install with their import names if different
packages = [
    ("datasets", "datasets"),
    ("pandas", "pandas"),
    ("llama-index", "llama_index"),
    ("chromadb", "chromadb"),
    ("openai", "openai"),
]

# Install each package
for package_name, import_name in packages:
    install_package(package_name, import_name)

# Check for `utils` module, typically custom or local
try:
    import utils
except ImportError:
    print("The 'utils' module is not found. Make sure it's available in your environment or install it manually.")


datasets not found. Installing...
pandas is already installed.
llama-index not found. Installing...
chromadb not found. Installing...
openai is already installed.
The 'utils' module is not found. Make sure it's available in your environment or install it manually.


In [2]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/ARGOG')
import utils




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


ModuleNotFoundError: No module named 'utils'

In [1]:
# Importing necessary libraries for loading datasets, data manipulation, document processing, vector storage, and embeddings.
from datasets import load_dataset
from datasets import Dataset
import pandas as pd
from llama_index.core import Document, StorageContext, VectorStoreIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceWindowNodeParser
import chromadb
from llama_index.core.node_parser import TokenTextSplitter
from utils import chunked_iterable, load_config
from llama_index.vector_stores.chroma import ChromaVectorStore
import openai
import os
from utils import *

ModuleNotFoundError: No module named 'datasets'